In [0]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [0]:
import urllib.request
urllib.request.urlretrieve("https://data.seattle.gov/resource/tazs-3rd5.csv?$limit=1200000", "/tmp/my.csv")
dbutils.fs.mv("file:/tmp/my.csv", "dbfs:/seattle_crime_analysis/data/sea_crime.csv")
display(dbutils.fs.ls("dbfs:/seattle_crime_analysis/data/"))

path,name,size,modificationTime
dbfs:/seattle_crime_analysis/data/sea_crime.csv,sea_crime.csv,295137527,1740171119000


In [0]:
data_path = "dbfs:/seattle_crime_analysis/data/sea_crime.csv"

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sea_crime_df = spark.read.format("csv").option("header", "true").load(data_path)
display(sea_crime_df.limit(10))
sea_crime_df.createOrReplaceTempView("sea_crime")

report_number,offense_id,offense_start_datetime,offense_end_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,precinct,sector,beat,mcpp,_100_block_address,longitude,latitude
2020-044452,12605598696,2020-02-03T08:00:00.000,2020-02-04T08:00:00.000,2020-02-05T10:06:28.000,A,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,J3,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399063,47.675117890
2020-044465,12605567653,2020-02-02T20:30:00.000,2020-02-02T21:30:00.000,2020-02-05T09:39:33.000,A,PROPERTY,ROBBERY,Robbery,120,N,U,U3,ROOSEVELT/RAVENNA,26TH AVE NE / NE BLAKELEY ST,-122.299552180,47.666384070
2020-044225,12605174036,2020-02-05T01:17:00.000,2020-02-05T02:21:00.000,2020-02-05T03:30:55.000,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,W,Q,Q1,MAGNOLIA,22XX BLOCK OF W RAYE ST,-122.384864805,47.642927340
2020-044076,12605081469,2020-02-05T00:51:21.000,null,2020-02-05T00:51:31.000,B,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,B,B2,BALLARD SOUTH,NW 46TH ST / 8TH AVE NW,-122.366195342,47.662193080
2020-044142,12605077150,2020-02-04T22:59:00.000,null,2020-02-05T00:45:08.000,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,M,M1,DOWNTOWN COMMERCIAL,14XX BLOCK OF 2ND AVE,-122.338449515,47.608788660
2020-044104,12605029468,2020-02-04T20:30:00.000,2020-02-04T20:45:00.000,2020-02-04T23:31:52.000,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,N,J,J3,PHINNEY RIDGE,71XX BLOCK OF GREENWOOD AVE N,-122.355309065,47.680526950
2020-043988,12604995759,2020-02-04T19:58:00.000,null,2020-02-04T22:46:19.000,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,SW,F,F2,ROXHILL/WESTWOOD/ARBOR HEIGHTS,26XX BLOCK OF SW BARTON ST,-122.367311993,47.521027580
2020-043928,12604963106,2020-02-04T19:01:00.000,null,2020-02-04T21:59:07.000,B,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,L,L2,NORTHGATE,15TH AVE NE / NE 95TH ST,-122.312077769,47.697616170
2020-044065,12605008517,2020-02-04T21:00:00.000,2020-02-04T21:15:00.000,2020-02-04T21:47:32.000,A,PROPERTY,LARCENY-THEFT,Theft From Motor Vehicle,23F,E,E,E2,CAPITOL HILL,15XX BLOCK OF 12TH AVE,-122.316845012,47.614683850
2020-044038,12604928711,2020-02-04T20:57:00.000,null,2020-02-04T21:20:35.000,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,E,E,E2,CAPITOL HILL,15XX BLOCK OF BROADWAY,-122.320789514,47.614655490


### 1. Counting the number of crims for different category

In [0]:
count_result = sea_crime_df.groupBy('offense_parent_group').count().orderBy('count', ascending=False)
display(count_result)

offense_parent_group,count
LARCENY-THEFT,434101
ASSAULT OFFENSES,169334
BURGLARY/BREAKING&ENTERING,133721
DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,110111
MOTOR VEHICLE THEFT,82174
FRAUD OFFENSES,65078
TRESPASS OF REAL PROPERTY,35445
DRUG/NARCOTIC OFFENSES,29516
ROBBERY,27417
DRIVING UNDER THE INFLUENCE,20574


Databricks visualization. Run in Databricks to view.

### 2. Counts the number of crimes for different district

In [0]:
loc_counts = sea_crime_df.groupBy("mcpp").count().orderBy('count', ascending=False)
display(loc_counts)


mcpp,count
DOWNTOWN COMMERCIAL,91534
CAPITOL HILL,72277
NORTHGATE,65864
QUEEN ANNE,60090
SLU/CASCADE,52159
UNIVERSITY,47121
ROOSEVELT/RAVENNA,42654
BALLARD SOUTH,38047
FIRST HILL,36502
LAKECITY,32386


Databricks visualization. Run in Databricks to view.

### 3. Analysis the number of crime in each month of 2015, 2016, 2017, 2018. Then give insight about business impact for result.

In [0]:
from pyspark.sql.functions import col, to_timestamp, date_format, year, month, hour, to_date
sea_crime_df = sea_crime_df.withColumn(
    "formatted_report_datetime", 
    date_format(to_timestamp(col("report_datetime"), "yyyy-MM-dd'T'HH:mm:ss.SSS"), "yyyy-MM-dd HH:mm:ss")
) \
    .withColumn(
    "IncidentDate", 
    date_format(to_timestamp(col("report_datetime"), "yyyy-MM-dd'T'HH:mm:ss.SSS"), "MM/dd/yyyy")
)

sea_crime_df = sea_crime_df.withColumn("year", year(col('formatted_report_datetime'))) \
    .withColumn("month", month(col('formatted_report_datetime'))) \
    .withColumn("time", hour(col('formatted_report_datetime')))
display(sea_crime_df.limit(10))


report_number,offense_id,offense_start_datetime,offense_end_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,precinct,sector,beat,mcpp,_100_block_address,longitude,latitude,formatted_report_datetime,IncidentDate,year,month,time
2020-044452,12605598696,2020-02-03T08:00:00.000,2020-02-04T08:00:00.000,2020-02-05T10:06:28.000,A,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,J3,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399063,47.675117890,2020-02-05 10:06:28,02/05/2020,2020,2,10
2020-044465,12605567653,2020-02-02T20:30:00.000,2020-02-02T21:30:00.000,2020-02-05T09:39:33.000,A,PROPERTY,ROBBERY,Robbery,120,N,U,U3,ROOSEVELT/RAVENNA,26TH AVE NE / NE BLAKELEY ST,-122.299552180,47.666384070,2020-02-05 09:39:33,02/05/2020,2020,2,9
2020-044225,12605174036,2020-02-05T01:17:00.000,2020-02-05T02:21:00.000,2020-02-05T03:30:55.000,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,W,Q,Q1,MAGNOLIA,22XX BLOCK OF W RAYE ST,-122.384864805,47.642927340,2020-02-05 03:30:55,02/05/2020,2020,2,3
2020-044076,12605081469,2020-02-05T00:51:21.000,null,2020-02-05T00:51:31.000,B,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,B,B2,BALLARD SOUTH,NW 46TH ST / 8TH AVE NW,-122.366195342,47.662193080,2020-02-05 00:51:31,02/05/2020,2020,2,0
2020-044142,12605077150,2020-02-04T22:59:00.000,null,2020-02-05T00:45:08.000,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,W,M,M1,DOWNTOWN COMMERCIAL,14XX BLOCK OF 2ND AVE,-122.338449515,47.608788660,2020-02-05 00:45:08,02/05/2020,2020,2,0
2020-044104,12605029468,2020-02-04T20:30:00.000,2020-02-04T20:45:00.000,2020-02-04T23:31:52.000,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,N,J,J3,PHINNEY RIDGE,71XX BLOCK OF GREENWOOD AVE N,-122.355309065,47.680526950,2020-02-04 23:31:52,02/04/2020,2020,2,23
2020-043988,12604995759,2020-02-04T19:58:00.000,null,2020-02-04T22:46:19.000,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,SW,F,F2,ROXHILL/WESTWOOD/ARBOR HEIGHTS,26XX BLOCK OF SW BARTON ST,-122.367311993,47.521027580,2020-02-04 22:46:19,02/04/2020,2020,2,22
2020-043928,12604963106,2020-02-04T19:01:00.000,null,2020-02-04T21:59:07.000,B,SOCIETY,DRIVING UNDER THE INFLUENCE,Driving Under the Influence,90D,N,L,L2,NORTHGATE,15TH AVE NE / NE 95TH ST,-122.312077769,47.697616170,2020-02-04 21:59:07,02/04/2020,2020,2,21
2020-044065,12605008517,2020-02-04T21:00:00.000,2020-02-04T21:15:00.000,2020-02-04T21:47:32.000,A,PROPERTY,LARCENY-THEFT,Theft From Motor Vehicle,23F,E,E,E2,CAPITOL HILL,15XX BLOCK OF 12TH AVE,-122.316845012,47.614683850,2020-02-04 21:47:32,02/04/2020,2020,2,21
2020-044038,12604928711,2020-02-04T20:57:00.000,null,2020-02-04T21:20:35.000,A,PROPERTY,LARCENY-THEFT,Shoplifting,23C,E,E,E2,CAPITOL HILL,15XX BLOCK OF BROADWAY,-122.320789514,47.614655490,2020-02-04 21:20:35,02/04/2020,2020,2,21


In [0]:
sea_crime_df.createOrReplaceTempView("sea_crime_df")
sea_crime_1518 = spark.sql("""
    SELECT year, month, COUNT(*) AS count 
    FROM sea_crime_df 
    WHERE year >= 2015
    AND year <= 2018 
    GROUP BY year, month 
    ORDER BY count DESC""")
display(sea_crime_1518)

year,month,count
2018,5,6667
2018,7,6628
2017,8,6506
2018,8,6466
2017,10,6458
2017,7,6440
2018,10,6396
2018,6,6380
2018,12,6275
2018,9,6247


Databricks visualization. Run in Databricks to view.

### Crime Distribution
#### 1. Crime Distribution Over Time
High Crime Rate Months:

May: May 2018 (6,667 crimes) and May 2016 (6,169 crimes) had the highest crime rates.

July: July 2018 (6,628 crimes) and July 2016 (6,125 crimes) also saw high crime activity.

August: August 2017 (6,506 crimes) and August 2018 (6,466 crimes) were peak months for crime.

Low Crime Months:

February: February 2018 (5,636 crimes) and February 2015 (5,016 crimes) had the lowest crime rates.

January: January 2015 (5,476 crimes) and January 2016 (6,072 crimes) also saw relatively low crime activity.

#### 2. Annual Trends
2018: Overall crime rates were high, especially in May, July, August, and October.

2017: Crime rates were also high, particularly in August, October, and July.

2016: Crime rates were relatively stable, with peaks in May and July.

2015: Crime rates were generally lower, especially in February and January.

#### 3. Seasonal Trends
Spring and Summer (May - August): Crime rates were consistently higher, possibly due to warmer weather and increased outdoor activities.

Fall and Winter (October - December): Crime rates remained high, potentially linked to holidays like Halloween and Christmas.

Winter (January - February): Crime rates were lower, likely due to colder weather and reduced outdoor activities.

###Business Impact
#### 1. Resource Allocation
Peak Season Resource Deployment:

Increase police patrols during high-crime months (e.g., May, July, August) in crime hotspots like city centers and commercial areas.

Enhance community policing and public safety campaigns to raise awareness among residents.

Low Season Resource Optimization:

Use low-crime months (e.g., January, February) for officer training, equipment maintenance, and resource optimization.

#### 2. Preventive Measures
Seasonal Prevention Strategies:

During spring and summer, increase nighttime patrols and surveillance in high-risk areas like entertainment districts and parks.

During holidays, deploy additional police forces to prevent theft, robbery, and other crimes.

Data-Driven Decision Making:

Use historical crime data to predict future trends and implement proactive measures.

Identify crime hotspots through data analysis and implement targeted security measures.

## 4. Analyse time frequency in Seattle districts and provide advice for local police department.

In [0]:
sea_crime_df.createOrReplaceTempView("sea_crime_df")
sea_crime_hournday = spark.sql("""
    SELECT IncidentDate, time, COUNT(*) AS count 
    FROM sea_crime_df 
    WHERE IncidentDate IN ('12/15/2020', '12/15/2021', '12/15/2022')
    GROUP BY IncidentDate, time
    ORDER BY count DESC""")
display(sea_crime_hournday)

IncidentDate,time,count
12/15/2022,6,54
12/15/2021,13,34
12/15/2021,14,34
12/15/2021,16,28
12/15/2021,10,19
12/15/2021,15,19
12/15/2020,10,18
12/15/2021,12,17
12/15/2021,17,15
12/15/2021,18,15


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import desc
top_3_districts = sea_crime_df.groupBy("mcpp") \
     .count() \
     .orderBy(desc("count")) \
     .limit(3)
display(top_3_districts)

mcpp,count
DOWNTOWN COMMERCIAL,91534
CAPITOL HILL,72277
NORTHGATE,65864


Databricks visualization. Run in Databricks to view.

In [0]:
crime_eventOf3dist = sea_crime_df.filter(sea_crime_df.mcpp.isin('DOWNTOWN COMMERCIAL', 'CAPITOL HILL', 'NORTHGATE')).groupby('crime_against_category', 'time').count().orderBy('crime_against_category', 'time')
display(crime_eventOf3dist)

crime_against_category,time,count
NOT_A_CRIME,2,1
NOT_A_CRIME,10,1
NOT_A_CRIME,11,1
NOT_A_CRIME,12,1
NOT_A_CRIME,19,1
PERSON,0,1828
PERSON,1,1869
PERSON,2,1913
PERSON,3,1177
PERSON,4,770


Databricks visualization. Run in Databricks to view.


#### 1. Crime Distribution by District
Top 3 High-Crime Districts:
Downtown Commercial: 91,534 crimes
Capitol Hill: 72,277 crimes
Northgate: 65,864 crimes
Advice:
Hotspot Policing: Focus on these high-crime districts with increased patrols, especially during peak crime hours.
Community Engagement: Work with local businesses and residents in these areas to develop community-based crime prevention strategies.
Infrastructure Improvements: Consider improving street lighting, installing surveillance cameras, and increasing police presence in these districts.
#### 2. Crime Distribution by Time of Day
Peak Crime Hours:
Property Crimes: Peak between 10 AM to 4 PM, with the highest number of crimes occurring around 1 PM.
Crimes Against Persons: Peak in the late afternoon and evening, with the highest number of crimes occurring between 5 PM to 9 PM.
Society-Related Crimes: Peak in the evening and late night, with the highest number of crimes occurring between 9 PM to 11 PM.
Advice:
Shift Adjustments: Adjust police shifts to ensure higher coverage during peak crime hours. For example, increase patrols in the late afternoon and evening when crimes against persons are more frequent.
Targeted Patrols: Focus on property crime prevention during the daytime, especially in commercial areas, and increase patrols in residential areas during the evening and night.
Public Awareness: Educate the public about safety measures during high-crime hours, such as avoiding walking alone at night and securing property during the day.